# Cadastro Geral

Atualmente há duas lista de cadastro geral no Ponto do Bem. O objetivo desse programa é conciliar os dados das duas em uma planilha só. Ademais, userei VBA para corresponder dados novos em futuras atualizaçôes.



In [1]:
import pandas as pd

In [22]:
geral_oficial = pd.read_csv("/content/CADASTRO_GERAL.csv", parse_dates=True)
geral_nao_oficial = pd.read_csv("/content/Cadastro_Geral_Nao_oficial.csv", parse_dates=True)

## Visualizações

In [23]:
geral_nao_oficial.head()

,Nº,Nome,Liberado?,Remédios / Roupas,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Tipo de Auxílio,Qtde de pessoas,(R$),Data de Atualização,Endereço,CPF,DATA DA PLANILHA,Observações
0,1.0,Abdaline Jean Baptiste,NaN,NaN,Tanque,98064-1512 - 97201-0948,3,2,2,não,NaN,7,NaN,6/19/2020,"Rua Almirante Melquíades de Souza, 322 Tanque ...",065.001.667-01,7/3/2020,NaN
1,2.0,Abigail Pereira Cabral,NaN,NaN,Praça Seca,998039-7091 (tel errado),1,3,NaN,não,NaN,4,NaN,6/5/2020,NaN,714.666.306-34,7/3/2020,NaN
2,3.0,Adélia Verônica Pinto da Silva,OK,Sim,Vila Isabel,99312-1634,1,2,0,não,"bolsa famila 130,00",3,NaN,4/20/2021,Rua viana drumond 38 casa 2 Vila Isabel,125.458..797-70,7/11/2020,NaN
3,4.0,Adelino da Silva Gomes,OK RECEBEU,NaN,Vila Isabel,98891-2516,3,1,NaN,NaN,NaN,4,NaN,NaN,"Rua Armando de Albuquerque, 33",011.833.607-03,6/27/2020,NaN
4,5.0,Aderaldo Araújo Simplício,NaN,NaN,Tijuca,99947-3523,NaN,NaN,NaN,NaN,NaN,0,NaN,7/25/2020,NaN,606.569.647-15,8/8/2020,NaN


In [20]:
geral_oficial.head()

,Nº+,Nome,EV/AB/PRAÇA,Pedidos de Roupa/Remédio,Bairro,Telefone,Crianças,Adultos,Idosos,Trabalha,Recebe Auxílio?,Qtde de pessoas,Renda mensal,Data de entrada,Endereço,CPF,CESTA 1,CESTA 2,CESTA 3,Última atualização,Observações
0,1,Abdaline Jean Baptiste,Praça,Não,Tanque,98064-1512,4,1,2,Não,Não,7,"1.100,00",???,Rua Almirante de Souza - Casa 322,065.001.667-01,NaN,NaN,NaN,22/03/2021,NaN
1,2,Abigail Pereira Cabral,NaN,NaN,Praça Seca,998039-7091 (tel errado),1,3,NaN,não,NaN,4,NaN,05/06/2020,NaN,714.666.306-34,NaN,NaN,NaN,03/07/2020,NaN
2,3,Adélia Verônica Pinto da Silva,AB,Sim,Vila Isabel,99312-1634,1,1,1,Não,Não,3,"0,00",11/08/2020,"Rua Senador Nabuco, 411",125.458.797-70,NaN,NaN,NaN,11/07/2020,NaN
3,4,Adelino da Silva Gomes,Praça,Sim,Vila Isabel,(21) 98891-2516,2,2,0,Não,Não,4,"0,00",29/05/2020,"Rua Armando de Albuquerque, 33",011.833.607-03,NaN,NaN,NaN,29/05/2020,NaN
4,5,Aderaldo Araújo Simplício,NaN,NaN,Tijuca,99947-3523,NaN,NaN,NaN,NaN,NaN,0,NaN,25/07/2020,NaN,606.569.647-15,NaN,NaN,NaN,08/08/2020,NaN


Vemos que a lista oficial tem mais dados do que a não oficial.

In [7]:
geral_nao_oficial.shape, geral_oficial.shape

((1368, 18), (1541, 21))

In [21]:
geral_oficial.dtypes

Nº+                          object
Nome                         object
EV/AB/PRAÇA                  object
Pedidos de Roupa/Remédio     object
Bairro                       object
Telefone                     object
Crianças                     object
Adultos                      object
Idosos                       object
Trabalha                     object
Recebe Auxílio?              object
Qtde de pessoas              object
Renda mensal                 object
Data de entrada              object
Endereço                     object
CPF                          object
CESTA 1                      object
CESTA 2                      object
CESTA 3                     float64
Última atualização           object
Observações                  object
dtype: object

A lista oficial foi toda reconhecida como string, exceto a coluna da Cesta 3. 

In [16]:
geral_nao_oficial.dtypes

Nº                     float64
Nome                    object
Liberado?               object
Remédios / Roupas       object
Bairro                  object
Telefone                object
Crianças                object
Adultos                 object
Idosos                  object
Trabalha                object
Tipo de Auxílio         object
Qtde de pessoas         object
(R$)                    object
Data de Atualização     object
Endereço                object
CPF                     object
DATA DA PLANILHA        object
Observações             object
dtype: object

## Modificando e validando dados

In [28]:
#Pegando o nome das colunas
colunas_cad_oficial = []
colunas_cad_nao_oficial = []

for col in geral_oficial.columns:
  colunas_cad_oficial.append(col)

for col in geral_nao_oficial.columns:
  colunas_cad_nao_oficial.append(col)

### Excluindo colunas desnecessárias

In [37]:
geral_nao_oficial = geral_nao_oficial.drop("Liberado?", axis = 1)

### Estruturando a Coluna "CPF"

In [53]:
# Essa função enviar cada valor para ser checado
def separacao(data_serie):

  for i in len(data_serie):
    valores = cpf(data_serie[i])
    
    if not valores[1]:
      data_serie[i] = valores[0]


In [ ]:
# Essa função checará se o CPF está no formato XXX.XXX.XXX-XX
def cpf(valor):

  if len(valor) == 14:
    return [valor, True]

  else:
    cpf = ["", "", "", ".", "", "", "", ".", "", "", "", "-", "", ""]
    valor = valor.split().strip() # Transforma em uma lista

    for i in range(len(valor), 0, -1):
      # Começa a verificar do ultimo ao primeiro

      if valor[i].isdigit():
        # Se o valor for digito

        if cpf[í] != "":
          # se no mesmos index não tiver vazio
          
          cpf[i + 1] = valor[i]
        
        else:
          cpf[i] = valor[i]
      
      else:


    
